Dan Shea  
2021-05-17  
#### Simulating civilizational lifetimes
I saw a fascinating talk by https://www.youtube.com/watch?v=LrrNu_m_9K4 which references this paper https://arxiv.org/abs/2010.12358

Kipping D., Frank A., Scharf C. (2020) Contact Inequality -- First Contact Will Likely Be With An Older Civilization

I decided to try my hand at whipping up a small simulation of my own. I briefly describe it below. The main intent was to construct something to play with `ipycanvas` in Jupyter Lab and to also see if I could draw similar conclusions _prior_ to reading the paper just by reasoning out a quick model.

Below I define a civilization as having a lifetime drawn from an exponential distribution scaled by 0.1 the life of the universe. 
I chose an exponential to sample lifetimes, since if we consider the death of a civilization as the expected event, we could then use an exponential to model a Poisson point process (_i.e._ - a process in which events occur continuously and independently at a constant average rate).

I re-paint a civilization red to signify if it lives beyond the median lifetime of all civilizations (_i.e._ - it is "old").  

The simulation runs for 100,000 epochs (_i.e._ - The simulated universe's lifetime) and updates the civilizations at each epoch.  
(You can change this, along with the number of civilizations modeled, etc. in the `Simulation` class below.)

_Note:_ The animation requires `ipycanvas` which may be obtained/installed by reading https://ipycanvas.readthedocs.io/en/latest/installation.html

It's a pretty neat way to add visualization to a notebook.

__TODO__:  
* Try using a `MultiCanvas` to track the proportion of "old" civilization to "young". It would be nice to display this value in the upper right corner.
* Find some time to read through the paper and compare my model to the one presented, see where I deviated and maybe perform a few runs to gather data and perform a couple more analyses.

In [1]:
import numpy as np
from ipycanvas import RoughCanvas, hold_canvas
import time

In [2]:
class Civilization:
    def __init__(self, epochs):
        self.lifetime = int(np.random.exponential() * (epochs/10))
        self.age = 0
        self.birth = np.random.choice(range(epochs))
        self.alive = False
        self.born  = False
        self.died  = False
    def update(self, now):
        # If we are born now, update our status
        if self.birth == now:
            self.born  = True
            self.alive = True
        # If we die now, update our status
        if self.birth + self.lifetime == now:
            self.alive = False
            self.died  = True
        # If we're alive update our age
        if self.alive:
            self.age += 1
    def __repr__(self):
        return f'Civilization(birth={self.birth}, lifetime={self.lifetime})'

In [11]:
class Simulation():
    def __init__(self):
        self.num_epochs = 100000
        self.now = 0
        self.num_civilizations = 100
        # Number of rows and columns in the grid
        self.num_rows = 10
        self.num_cols = 10
        # The radius of the civilization marker
        self.civ_radius = 25
        # The size of the roughCanvas used for visualization
        self.Grid_width = (self.num_rows * 2 * self.civ_radius) + 2 * self.civ_radius
        self.Grid_height = (self.num_rows * 2 * self.civ_radius) + 2 * self.civ_radius
        # Where the grid starts
        self.X_start = self.civ_radius
        self.Y_start = self.civ_radius
        
        # Initial pool of civilizations
        self.civilizations = [Civilization(self.num_epochs) for i in range(self.num_rows*self.num_cols)]
        self.median_lifetime = np.median(np.array([c.lifetime for c in self.civilizations]))
        
    def step(self, canvas):
        idx = 0
        for c in self.civilizations:
            draw=False
            # update the civilization
            c.update(self.now)
            # If the civilization is alive it is to be drawn
            if c.alive:
                canvas.fill_style = 'blue'
                canvas.rough_fill_style = 'hachure'
                draw=True
            # If the civilization is considered old, it is drawn differently
            if c.alive and (c.age >= self.median_lifetime):
                canvas.fill_style = 'red'
                canvas.rough_fill_style = 'zigzag'
                draw=True
            # If the civilization is dead, it is drawn differently
            if c.died:
                canvas.fill_style = 'black'
                canvas.rough_fill_style = 'cross-hatch'
                draw=True
            if draw:
                x_pos = idx % self.num_cols
                y_pos = idx // self.num_cols
                x_draw = 2*self.civ_radius * (x_pos+1)
                y_draw = 2*self.civ_radius * (y_pos+1)
                canvas.fill_circle(x_draw, y_draw, self.civ_radius)
            idx += 1
        self.now += 1

In [12]:
sim = Simulation()
canvas = RoughCanvas(width=sim.Grid_width, height=sim.Grid_height)
canvas.roughness = 2
canvas.line_width = 2
display(canvas)

RoughCanvas(height=550, width=550)

In [ ]:
for i in range(sim.num_epochs):
    with hold_canvas(canvas):
        canvas.clear()
        sim.step(canvas)
        time.sleep(0.02)
        